In [42]:
import pandas as pd
import collections

# CONFIG = True then و will be splitted into its own word
# Note: Currently, this will also split wrongly words like وقب	to و and قب	
SPLIT_WAW = True

with open('quran-text/quran-uthmani.txt') as f:
    quran_text = [l.strip() for l in f if not l.strip().startswith('#') and l.strip()]

In [2]:
FILTERED_CHARS = \
    '|123456789 \t\nًٌَُّْ	' \
    'ِ ۫ ۨ ۬ ۬' \
    '۪ ۣۭۜ۠' \
    '	' \
    'ٰ'
def filter_char(text):
    return ''.join(c for c in text if c not in FILTERED_CHARS)

# split basmallah to its own
i = 0 
while i < len(quran_text):
    aya = quran_text[i].split('|')
    if aya[2].startswith('بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ '):
        quran_text.insert(i, f'{aya[0]}|0|بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ')
        i += 1
        quran_text[i] = f'{aya[0]}|1|{aya[2][38:]}'
    i += 1

df = pd.DataFrame({'sura': int(a[0]), 'verse': int(a[1]), 'text': a[2]} for a in [a.split('|') for a in quran_text])
df

,sura,verse,text
0,1,1,بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ
1,1,2,ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَٰلَمِينَ
2,1,3,ٱلرَّحْمَٰنِ ٱلرَّحِيمِ
3,1,4,مَٰلِكِ يَوْمِ ٱلدِّينِ
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...
6341,114,2,مَلِكِ ٱلنَّاسِ
6342,114,3,إِلَٰهِ ٱلنَّاسِ
6343,114,4,مِن شَرِّ ٱلْوَسْوَاسِ ٱلْخَنَّاسِ
6344,114,5,ٱلَّذِى يُوَسْوِسُ فِى صُدُورِ ٱلنَّاسِ


In [45]:
def split_filter_word(text):
    words = [w for w in [filter_char(w) for w in text.split(' ')] if w.strip()]
    if SPLIT_WAW:
        i = 0
        while i < len(words):
            if words[i].startswith('و'):
                words.insert(i, 'و')
                i += 1
                words[i] = words[i][1:]
            i += 1

    return words


df['words'] = df.text.apply(split_filter_word)
df['word_count'] = df.words.apply(lambda w: len(w))

In [4]:
df.head(10)

,sura,verse,text,words,word_count
0,1,1,بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ,"[بسم, ٱلله, ٱلرحمن, ٱلرحيم]",4
1,1,2,ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَٰلَمِينَ,"[ٱلحمد, لله, رب, ٱلعلمين]",4
2,1,3,ٱلرَّحْمَٰنِ ٱلرَّحِيمِ,"[ٱلرحمن, ٱلرحيم]",2
3,1,4,مَٰلِكِ يَوْمِ ٱلدِّينِ,"[ملك, يوم, ٱلدين]",3
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,"[إياك, نعبد, وإياك, نستعين]",4
5,1,6,ٱهْدِنَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ,"[ٱهدنا, ٱلصرط, ٱلمستقيم]",3
6,1,7,صِرَٰطَ ٱلَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...,"[صرط, ٱلذين, أنعمت, عليهم, غير, ٱلمغضوب, عليهم...",9
7,2,0,بِسْمِ ٱللَّهِ ٱلرَّحْمَٰنِ ٱلرَّحِيمِ,"[بسم, ٱلله, ٱلرحمن, ٱلرحيم]",4
8,2,1,الٓمٓ,[الٓمٓ],1
9,2,2,ذَٰلِكَ ٱلْكِتَٰبُ لَا رَيْبَ فِيهِ هُدًى لِّل...,"[ذلك, ٱلكتب, لا, ريب, فيه, هدى, للمتقين]",7


In [46]:
# longest verse by word count
df.sort_values('word_count', ascending=False).head(10)

,sura,verse,text,words,word_count
289,2,282,يَٰٓأَيُّهَا ٱلَّذِينَ ءَامَنُوٓا۟ إِذَا تَدَا...,"[يٓأيها, ٱلذين, ءامنوٓا۟, إذا, تداينتم, بدينٍ,...",147
507,4,12,وَلَكُمْ نِصْفُ مَا تَرَكَ أَزْوَٰجُكُمْ إِن ل...,"[و, لكم, نصف, ما, ترك, أزوجكم, إن, لم, يكن, له...",102
5565,73,20,إِنَّ رَبَّكَ يَعْلَمُ أَنَّكَ تَقُومُ أَدْنَى...,"[إن, ربك, يعلم, أنك, تقوم, أدنى, من, ثلثى, ٱلي...",91
109,2,102,وَٱتَّبَعُوا۟ مَا تَتْلُوا۟ ٱلشَّيَٰطِينُ عَلَ...,"[و, ٱتبعوا۟, ما, تتلوا۟, ٱلشيطين, على, ملك, سل...",86
2843,24,31,وَقُل لِّلْمُؤْمِنَٰتِ يَغْضُضْنَ مِنْ أَبْصَٰ...,"[و, قل, للمؤمنت, يغضضن, من, أبصرهن, و, يحفظن, ...",85
506,4,11,يُوصِيكُمُ ٱللَّهُ فِىٓ أَوْلَٰدِكُمْ لِلذَّكَ...,"[يوصيكم, ٱلله, فىٓ, أولدكم, للذكر, مثل, حظ, ٱل...",80
203,2,196,وَأَتِمُّوا۟ ٱلْحَجَّ وَٱلْعُمْرَةَ لِلَّهِ فَ...,"[و, أتموا۟, ٱلحج, و, ٱلعمرة, لله, فإن, أحصرتم,...",79
2873,24,61,لَّيْسَ عَلَى ٱلْأَعْمَىٰ حَرَجٌ وَلَا عَلَى ٱ...,"[ليس, على, ٱلأعمى, حرج, و, لا, على, ٱلأعرج, حر...",79
448,3,154,ثُمَّ أَنزَلَ عَلَيْكُم مِّنۢ بَعْدِ ٱلْغَمِّ ...,"[ثم, أنزل, عليكم, منۢ, بعد, ٱلغم, أمنة, نعاسا,...",79
3616,33,53,يَٰٓأَيُّهَا ٱلَّذِينَ ءَامَنُوا۟ لَا تَدْخُلُ...,"[يٓأيها, ٱلذين, ءامنوا۟, لا, تدخلوا۟, بيوت, ٱل...",77


In [47]:
# shortest verse by word count
df.sort_values('word_count').head(10)

,sura,verse,text,words,word_count
295,3,1,الٓمٓ,[الٓمٓ],1
3498,31,1,الٓمٓ,[الٓمٓ],1
4516,45,1,حمٓ,[حمٓ],1
2956,26,1,طسٓمٓ,[طسٓمٓ],1
4456,44,1,حمٓ,[حمٓ],1
3533,32,1,الٓمٓ,[الٓمٓ],1
2267,19,1,كٓهيعٓصٓ,[كٓهيعٓصٓ],1
3437,30,1,الٓمٓ,[الٓمٓ],1
4366,43,1,حمٓ,[حمٓ],1
4313,42,2,عٓسٓقٓ,[عٓسٓقٓ],1


In [48]:
df_words = df[['sura','verse','words']].explode('words').rename(columns={'words': 'word'})
df_words

,sura,verse,word
0,1,1,بسم
0,1,1,ٱلله
0,1,1,ٱلرحمن
0,1,1,ٱلرحيم
1,1,2,ٱلحمد
...,...,...,...
6344,114,5,ٱلناس
6345,114,6,من
6345,114,6,ٱلجنة
6345,114,6,و


In [49]:
#longest word in the quran
df_words['wordlen'] = df_words['word'].apply(len)
df_words.sort_values('wordlen', ascending=False).head(10)

,sura,verse,word,wordlen
2871,24,59,فليستـٔذنوا۟,12
1153,7,194,فليستجيبوا۟,11
193,2,186,فليستجيبوا۟,11
578,4,83,يستنۢبطونهۥ,11
4262,41,6,فٱستقيموٓا۟,11
4491,44,36,بـٔابآئنآ,11
2174,18,19,ليتسآءلوا۟,11
1424,10,53,يستنۢبـٔونك,11
1838,15,24,ٱلمستـٔخرين,11
4540,45,25,بـٔابآئنآ,11


In [50]:
df_word_occurence = df_words[['word','verse']].groupby('word').count().rename(columns={'verse': 'count'}).reset_index()
# most occuring word
df_word_occurence.sort_values('count', ascending=False).head(10)

,word,count
11084,و,10074
10252,من,2926
13977,ٱلله,2505
9561,ما,1385
8568,لا,1354
1606,إن,1220
7556,فى,1125
13476,ٱلذين,975
1217,أن,705
6346,على,640


In [51]:
# word that has the same occurences
df_word_same_occurence = df_word_occurence.groupby('count').agg(list)
df_word_same_occurence = df_word_same_occurence[df_word_same_occurence['word'].apply(len) > 1]
df_word_same_occurence.sort_index(ascending=False).head(20)

,word
count,
225,"[لم, لهۥ]"
179,"[منهم, منۢ]"
165,"[إنهۥ, عليكم]"
143,"[إنا, لله]"
127,"[بل, فيه]"
116,"[لآ, يشآء]"
111,"[ربهم, مما]"
110,"[ٱلحق, ٱلدنيا]"
89,"[فمن, منه]"


,sura,verse,word,wordlen
4,1,5,وإياك,5
6,1,7,ولا,3
10,2,3,ويقيمون,7
10,2,3,ومما,4
11,2,4,وٱلذين,6
...,...,...,...,...
6336,113,3,ومن,3
6336,113,3,وقب,3
6337,113,4,ومن,3
6338,113,5,ومن,3
